In [1]:
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
def query_combined_data(client, dataset_id, table_id, print_cost=False):
    """
    Execute a combined query to get distinct sensors, total entries, and last timestamp grouped by ExperimentData_Exp_name.

    Parameters:
        client (bigquery.Client): Initialized BigQuery client.
        dataset_id (str): The ID of the dataset.
        table_id (str): The ID of the table.
        print_cost (bool): Flag to print query cost.

    Returns:
        tuple: (query results, estimated cost in USD)
    """
    # Define the combined SQL query
    query = f"""
    SELECT
      ExperimentData_Exp_name,
      COUNT(DISTINCT SensorData_Name) AS sensor_count,
      COUNT(*) AS num_entries,
      MAX(TimeStamp) AS last_timestamp
    FROM
      `iucc-f4d.{dataset_id}.{table_id}`
    GROUP BY
      ExperimentData_Exp_name
    ORDER BY
      ExperimentData_Exp_name;
    """

    # Configure the query for a dry run to estimate costs
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    query_job = client.query(query, job_config=job_config)

    # Calculate estimated cost
    bytes_processed = query_job.total_bytes_processed
    estimated_cost = (bytes_processed / (1024 ** 4)) * 5  # Cost in USD ($5 per TB)

    if print_cost:
        print(f"Estimated cost for table {table_id} in dataset {dataset_id}: ${estimated_cost:.4f}")

    # Execute the actual query
    query_job = client.query(query)
    return query_job.result(), estimated_cost

In [3]:
def query_daily_data(client, dataset_id, table_id):
    """
    Query to calculate the amount of data found per day for each dataset_id.

    Parameters:
        client (bigquery.Client): Initialized BigQuery client.
        dataset_id (str): The ID of the dataset.
        table_id (str): The ID of the table.

    Returns:
        pd.DataFrame: DataFrame containing daily data counts.
    """
    query = f"""
    SELECT
      DATE(TimeStamp) AS date,
      COUNT(*) AS num_entries
    FROM
      `iucc-f4d.{dataset_id}.{table_id}`
    GROUP BY
      date
    ORDER BY
      date;
    """

    query_job = client.query(query)
    results = query_job.result()

      # Calculate estimated cost
    bytes_processed = query_job.total_bytes_processed
    estimated_cost = (bytes_processed / (1024 ** 4)) * 5  # Cost in USD ($5 per TB)


    # Convert query results to a DataFrame
    data = []
    for row in results:
        data.append({
            "dataset_id": dataset_id,
            "date": row.date,
            "num_entries": row.num_entries
        })

    return pd.DataFrame(data),estimated_cost

In [4]:
def create_daily_plot(df):
    """
    Create a plot of the amount of data per day for each dataset_id.

    Parameters:
        df (pd.DataFrame): The DataFrame containing daily data counts.
    """
    plt.figure(figsize=(12, 8))

    for dataset_id, group in df.groupby('dataset_id'):
        plt.plot(group['date'], group['num_entries'], marker='o', label=dataset_id)

    plt.xlabel('Date')
    plt.ylabel('Number of Entries')
    plt.title('Daily Data Counts per Dataset')
    plt.legend(title='Dataset ID', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [5]:

# Path to your credentials JSON file
credentials_path = "read_BQ.json"

# Load credentials from the JSON file
credentials = service_account.Credentials.from_service_account_file(credentials_path)

# Initialize BigQuery client
client = bigquery.Client(credentials=credentials, project="iucc-f4d")

# Initialize an empty list to store data for the DataFrame
data = []

dataset_costs = {}  # Dictionary to store costs per dataset
combined_cost = 0   # Variable to track the combined cost

In [6]:
# Iterate through datasets and tables, then run the query for each
for dataset in client.list_datasets():
    dataset_id = dataset.dataset_id
    dataset_total_cost = 0

    for table in client.list_tables(dataset_id):
        table_id = table.table_id
        print(f"Running query for dataset: {dataset_id}, table: {table_id}")
        results, estimated_cost = query_combined_data(client, dataset_id, table_id, print_cost=True)

        # Add query results to the data list
        for row in results:
            data.append({
                "dataset_id": dataset_id,
                "table_id": table_id,
                "ExperimentData_Exp_name": row.ExperimentData_Exp_name,
                "sensor_count": row.sensor_count,
                "num_entries": row.num_entries,
                "last_timestamp": row.last_timestamp,
                "estimated_cost": estimated_cost
            })

        dataset_total_cost += estimated_cost

    # Store total cost for the dataset
    dataset_costs[dataset_id] = dataset_total_cost
    combined_cost += dataset_total_cost

    print(f"Total estimated cost for dataset {dataset_id}: ${dataset_total_cost:.4f}")

# Print combined cost for all datasets
print(f"Combined estimated cost for all datasets: ${combined_cost:.10f}")

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

Running query for dataset: GrowthRoom, table: d83adde2608f
Estimated cost for table d83adde2608f in dataset GrowthRoom: $0.0000
Total estimated cost for dataset GrowthRoom: $0.0000
Running query for dataset: Icore_Pi, table: 2ccf6730ab5f
Estimated cost for table 2ccf6730ab5f in dataset Icore_Pi: $0.0000
Total estimated cost for dataset Icore_Pi: $0.0000
Running query for dataset: Yakir, table: d83adde260d1
Estimated cost for table d83adde260d1 in dataset Yakir: $0.0000
Total estimated cost for dataset Yakir: $0.0000
Running query for dataset: cellolar_developer_room, table: 2ccf6730ab8c
Estimated cost for table 2ccf6730ab8c in dataset cellolar_developer_room: $0.0000
Total estimated cost for dataset cellolar_developer_room: $0.0000
Running query for dataset: developer_room, table: d83adde26159
Estimated cost for table d83adde26159 in dataset developer_room: $0.0000
Total estimated cost for dataset developer_room: $0.0000
Combined estimated cost for all datasets: $0.0000872688


In [7]:
# Initialize an empty list to store data for the DataFrame
daily_data = []
conmbined_cost = 0
# Iterate through datasets and tables, then run the daily data query for each
for dataset in client.list_datasets():
    dataset_id = dataset.dataset_id

    for table in client.list_tables(dataset_id):
        table_id = table.table_id
        daily_df, estimated_cost = query_daily_data(client, dataset_id, table_id)
        print(f"Running daily data query for dataset: {dataset_id}, table: {table_id}, estimated cost: ${estimated_cost:.4f}")
        combined_cost += estimated_cost
        daily_data.append(daily_df)

# Combine all daily data into a single DataFrame
daily_data_df = pd.concat(daily_data, ignore_index=True)
print(f"Combined estimated cost for all datasets: ${combined_cost:.10f}")
# Display the DataFrame
print(daily_data_df)

Running daily data query for dataset: GrowthRoom, table: d83adde2608f, estimated cost: $0.0000
Running daily data query for dataset: Icore_Pi, table: 2ccf6730ab5f, estimated cost: $0.0000
Running daily data query for dataset: Yakir, table: d83adde260d1, estimated cost: $0.0000
Running daily data query for dataset: cellolar_developer_room, table: 2ccf6730ab8c, estimated cost: $0.0000
Running daily data query for dataset: developer_room, table: d83adde26159, estimated cost: $0.0000
Combined estimated cost for all datasets: $0.0000940909
         dataset_id        date  num_entries
0        GrowthRoom  2024-10-14         1853
1        GrowthRoom  2024-10-15         4435
2        GrowthRoom  2024-10-16          960
3        GrowthRoom  2024-10-17          960
4        GrowthRoom  2024-10-18          960
..              ...         ...          ...
212  developer_room  2025-01-04         1437
213  developer_room  2025-01-05         1455
214  developer_room  2025-01-06         1414
215  deve

In [8]:
# convert date to datetime
daily_data_df['date'] = pd.to_datetime(daily_data_df['date'])
# sort the data by date
daily_data_df = daily_data_df.sort_values(by=["dataset_id",'date'])
# calculate the comlutative sum of num_entries foer each dataset_id
daily_data_df["cumulative_num_entries"] = daily_data_df.groupby('dataset_id')['num_entries'].cumsum()

db_sum = 0
dicty_db_size = {}
for day in sorted(daily_data_df.date.unique()):
    # Filter the data for the current day
    day_data = daily_data_df[daily_data_df['date'] == day]
    # Create a plot of the data for the current day
    db_sum+=day_data["cumulative_num_entries"].sum()
    dicty_db_size[day] = db_sum
# create df of dicty_db_size
df_db_size = pd.DataFrame(list(dicty_db_size.items()),columns = ['date','db_size'])

In [9]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create subplots
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    subplot_titles=('Cumulative Number of Entries Over Time', 'Database Size Over Time'))

# Add the first plot (Cumulative Number of Entries Over Time)
for dataset_id in daily_data_df['dataset_id'].unique():
    dataset_df = daily_data_df[daily_data_df['dataset_id'] == dataset_id]
    fig.add_trace(go.Scatter(x=dataset_df['date'], y=dataset_df['cumulative_num_entries'], mode='lines+markers', name=dataset_id), row=1, col=1)

# Add the second plot (Database Size Over Time)
fig.add_trace(go.Scatter(x=df_db_size['date'], y=df_db_size['db_size'], mode='lines+markers', name='Database Size'), row=2, col=1)

# Update layout
fig.update_layout(height=800, width=1000, title_text="Data Analysis", template='plotly_white')
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Cumulative Number of Entries", row=1, col=1)
fig.update_yaxes(title_text="Database Size", row=2, col=1)

# Show the plot
fig.show()